In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'fourier'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_fourier_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,307.17,-0.02,0.00,-0.01,-0.00,0.01,0.00,-0.00,0.00,-0.00,...,0.00,0.00,0.00,-0.00,0.00,-0.0,0.00,0.00,0.00,-0.0
3,-0.02,229.87,0.00,0.00,0.01,0.01,-0.00,-0.00,0.00,-0.00,...,-0.00,-0.00,-0.00,-0.00,-0.00,0.0,0.00,0.00,-0.00,-0.0
4,0.00,0.00,191.35,0.01,-0.00,-0.00,0.00,-0.00,0.00,0.00,...,0.00,-0.00,-0.00,-0.00,0.00,-0.0,-0.00,0.00,0.00,0.0
5,-0.01,0.00,0.01,147.67,0.00,-0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,-0.00,-0.00,-0.00,0.0,-0.00,0.00,-0.00,0.0
6,-0.00,0.01,-0.00,0.00,118.31,0.00,0.00,-0.00,-0.00,-0.00,...,-0.00,0.00,-0.00,-0.00,0.00,-0.0,-0.00,0.00,0.00,0.0
7,0.01,0.01,-0.00,-0.00,0.00,90.66,0.00,-0.00,0.00,0.00,...,-0.00,-0.00,-0.00,-0.00,0.00,0.0,-0.00,-0.00,-0.00,0.0
8,0.00,-0.00,0.00,0.00,0.00,0.00,72.38,0.00,0.00,0.00,...,-0.00,0.00,0.00,0.00,-0.00,-0.0,-0.00,0.00,-0.00,-0.0
9,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,58.07,-0.00,0.00,...,0.00,-0.00,-0.00,-0.00,-0.00,-0.0,-0.00,0.00,0.00,-0.0
10,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,45.64,0.00,...,0.00,-0.00,0.00,-0.00,0.00,-0.0,0.00,-0.00,0.00,0.0
11,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,36.21,...,0.00,-0.00,0.00,-0.00,-0.00,0.0,-0.00,0.00,-0.00,-0.0


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,1.00000,-0.00006,0.00001,-0.00003,-0.00002,0.00003,0.00003,-0.00003,0.00002,-0.00000,...,0.00002,0.00001,0.00002,-0.00002,0.00000,-0.00000,0.00002,0.00001,0.00002,-0.00000
3,-0.00006,1.00000,0.00000,0.00002,0.00003,0.00005,-0.00002,-0.00003,0.00003,-0.00000,...,-0.00002,-0.00002,-0.00002,-0.00001,-0.00001,0.00000,0.00000,0.00001,-0.00000,-0.00001
4,0.00001,0.00000,1.00000,0.00007,-0.00003,-0.00001,0.00002,-0.00001,0.00001,0.00000,...,0.00000,-0.00000,-0.00001,-0.00002,0.00001,-0.00001,-0.00000,0.00001,0.00000,0.00001
5,-0.00003,0.00002,0.00007,1.00000,0.00004,-0.00002,0.00002,0.00005,0.00001,0.00000,...,0.00001,0.00002,-0.00001,-0.00002,-0.00002,0.00001,-0.00000,0.00001,-0.00000,0.00000
6,-0.00002,0.00003,-0.00003,0.00004,1.00000,0.00000,0.00003,-0.00001,-0.00002,-0.00000,...,-0.00000,0.00001,-0.00000,-0.00001,0.00001,-0.00000,-0.00000,0.00001,0.00000,0.00000
7,0.00003,0.00005,-0.00001,-0.00002,0.00000,1.00000,0.00002,-0.00000,0.00001,0.00000,...,-0.00004,-0.00000,-0.00000,-0.00001,0.00002,0.00001,-0.00003,-0.00002,-0.00001,0.00001
8,0.00003,-0.00002,0.00002,0.00002,0.00003,0.00002,1.00000,0.00007,0.00004,0.00000,...,-0.00003,0.00000,0.00001,0.00000,-0.00004,-0.00001,-0.00000,0.00000,-0.00001,-0.00001
9,-0.00003,-0.00003,-0.00001,0.00005,-0.00001,-0.00000,0.00007,1.00000,-0.00007,0.00001,...,0.00000,-0.00005,-0.00000,-0.00001,-0.00002,-0.00001,-0.00000,0.00000,0.00001,-0.00000
10,0.00002,0.00003,0.00001,0.00001,-0.00002,0.00001,0.00004,-0.00007,1.00000,0.00005,...,0.00000,-0.00000,0.00001,-0.00003,0.00001,-0.00000,0.00001,-0.00000,0.00000,0.00000
11,-0.00000,-0.00000,0.00000,0.00000,-0.00000,0.00000,0.00000,0.00001,0.00005,1.00000,...,0.00001,-0.00002,0.00001,-0.00001,-0.00002,0.00001,-0.00002,0.00000,-0.00001,-0.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.04282597861218367

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.95617881e+03 1.65421812e+03 1.14217744e+03 6.80044079e+02
 4.35528167e+02 2.55711096e+02 1.62692284e+02 1.04468864e+02
 6.44708272e+01 4.04981345e+01 2.47843165e+01 1.66131687e+01
 9.04325175e+00 5.88332080e+00 3.49075072e+00 2.05127514e+00
 1.17202416e+00 6.59774372e-01 3.60100857e-01 2.05179955e-01
 1.03819768e-01 5.19069183e-02 2.50953950e-02 1.19232595e-02
 5.08091958e-03 2.01808131e-03 7.53258440e-04 2.56343976e-04
 3.78069774e-05 5.91553385e-06 4.90559341e-07 5.82991008e-28]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.997774,-0.052431,-0.030606,-0.018671,-0.013448,-0.009551,-0.007362,-0.005813,-0.004481,-0.003533,...,-0.000088,-0.000061,-0.000040,-0.000027,-0.000016,-0.000010,-0.000004,-0.000002,-4.511669e-07,-1.362444e-07
1,0.048278,0.993913,-0.087934,-0.033738,-0.021342,-0.014134,-0.010613,-0.008192,-0.006236,-0.004887,...,-0.000123,-0.000086,-0.000056,-0.000036,-0.000022,-0.000013,-0.000005,-0.000002,-7.321081e-07,-2.059189e-07
2,0.032577,0.082513,0.992663,-0.067656,-0.034624,-0.020864,-0.014995,-0.011351,-0.008557,-0.006645,...,-0.000164,-0.000114,-0.000075,-0.000048,-0.000029,-0.000018,-0.000007,-0.000003,-9.416157e-07,-2.201452e-07
3,0.020452,0.035215,0.059755,0.992936,-0.081156,-0.034365,-0.022228,-0.015918,-0.011693,-0.008920,...,-0.000213,-0.000148,-0.000098,-0.000063,-0.000039,-0.000023,-0.000009,-0.000004,-1.275354e-06,-3.079070e-07
4,0.015813,0.024238,0.034439,0.073727,0.992433,-0.071310,-0.036347,-0.023833,-0.016629,-0.012347,...,-0.000285,-0.000198,-0.000131,-0.000084,-0.000051,-0.000031,-0.000012,-0.000005,-1.642099e-06,-4.153837e-07
5,0.011349,0.016318,0.021351,0.034077,0.062353,0.991819,-0.086066,-0.040799,-0.024979,-0.017510,...,-0.000378,-0.000260,-0.000171,-0.000109,-0.000067,-0.000040,-0.000016,-0.000007,-2.246548e-06,-4.866809e-07
6,0.009189,0.012887,0.016211,0.023570,0.035328,0.076360,0.990025,-0.092664,-0.041196,-0.025987,...,-0.000497,-0.000342,-0.000224,-0.000143,-0.000090,-0.000054,-0.000021,-0.000009,-3.003275e-06,-7.440722e-07
7,0.007661,0.010530,0.013006,0.017983,0.024948,0.041279,0.081759,0.989410,-0.089918,-0.042590,...,-0.000650,-0.000456,-0.000297,-0.000189,-0.000118,-0.000071,-0.000028,-0.000012,-3.741558e-06,-9.520848e-07
8,0.006096,0.008284,0.010139,0.013679,0.018124,0.026746,0.040915,0.077928,0.988917,-0.096495,...,-0.000853,-0.000593,-0.000389,-0.000250,-0.000153,-0.000093,-0.000036,-0.000015,-4.979988e-06,-1.228672e-06
9,0.005039,0.006807,0.008264,0.010972,0.014188,0.019878,0.027737,0.041905,0.084006,0.988073,...,-0.001129,-0.000787,-0.000515,-0.000329,-0.000204,-0.000122,-0.000048,-0.000020,-6.736217e-06,-1.670912e-06


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0022258367657147504,
 0.006086634490711784,
 0.0073368723775595335,
 0.007063861230214852,
 0.007567452016126741,
 0.008181466103342805,
 0.00997548115120761,
 0.010589859579247962,
 0.011082735920378783,
 0.011926515291012962,
 0.014612569051760005,
 0.014616799022317495,
 0.014432287713362135,
 0.017082109924759448,
 0.01625882680683799,
 0.0168011327783002,
 0.01747763353522236,
 0.018424266866518435,
 0.021024743850507344,
 0.022229140197485675,
 0.021369217083080816,
 0.023215066133383866,
 0.025837697279257,
 0.02754521699087864,
 0.028216849949562417,
 0.03136794099079954,
 0.03668050617542262,
 0.031122751089406542,
 0.024185362007784317,
 0.03236698564783225,
 0.04859848055859972,
 0.03343495812153374]